In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import copy
import plotly.express as px
import plotly.io as pio

#Pre-Processing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.model_selection import StratifiedKFold

#models
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
!pip install xgboost
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#Kernels
from sklearn.gaussian_process.kernels import RBF, WhiteKernel,Matern,ExpSineSquared,RationalQuadratic, ConstantKernel as C

#Scipy
from scipy.spatial import distance
import scipy
import scipy.optimize as opt
from scipy.optimize import differential_evolution
from scipy.optimize import basinhopping
from scipy.optimize import minimize
from scipy.stats import norm

#Pytorch
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim

#For plots
!pip install kaleido
!sudo apt-get install poppler-utils

#Optuna
!pip install optuna
import optuna


**Importing Data**

In [ ]:
!git clone https://github.com/ihdavjar/Data_For_Biomarker_Analysis.git

**Data Preparation for Lefse**

In [ ]:
data_1=pd.read_csv("/content/Data_For_Biomarker_Analysis/Main_Data/Otu_Counts.tsv")
data_1

In [ ]:
temp_l_cols=list(data_1.columns)
num_taxa_non_unique=[]
for i in range(1,len(temp_l_cols)-1):
  num_taxa_non_unique.append(sum(list(data_1[temp_l_cols[i]])))

plt.hist(num_taxa_non_unique)
plt.show()

num_taxa_non_unique=np.array(num_taxa_non_unique)
plt.hist(num_taxa_non_unique[num_taxa_non_unique<8000])
plt.show()

In [ ]:
org_data=pd.read_csv("/content/Data_For_Biomarker_Analysis/Main_Data/Otu_Counts.tsv")

taxonomy_arr=list(org_data["taxonomy"])
otu_ids=list(org_data["otu_id"])
print(taxonomy_arr)
print(otu_ids)

#Removing the taxonomy and the otu_ids from the feature table
org_data=org_data.drop(["otu_id","taxonomy"],axis=1)


temp_data=np.array(org_data)
temp_otus=[]
for i in range(len(temp_data)):
  temp_otus.append("Otu"+str(i+1))


#Adding the labels to feature table
org_data=org_data.T
org_data.columns=temp_otus
org_data['total_counts']=num_taxa_non_unique

print(org_data)

temp_data=pd.read_csv("/content/Data_For_Biomarker_Analysis/Main_Data/label_data.csv")
temp_data=temp_data.drop(['Unnamed: 0'],axis=1)
temp_data_np=np.array(temp_data)

print(temp_data)

dict_temp={}
for i in range(len(temp_data_np)):
  if (temp_data_np[i,1]=="Squamous cell carcinoma"):
    dict_temp[temp_data_np[i,0]]="Squamous"
  else:
    dict_temp[temp_data_np[i,0]]=temp_data_np[i,1]

l_temp_row=list(org_data.index)
temp_labels=[]

for i in range(len(l_temp_row)):
  if (dict_temp[l_temp_row[i]]=="Adenosquamous carcinoma"):
    org_data.drop([l_temp_row[i]],axis=0,inplace=True)
  else:
    temp_labels.append(dict_temp[l_temp_row[i]])


org_data_np=np.array(org_data)
org_data

print(len(num_taxa_non_unique))
org_data['label']=temp_labels


# org_data=org_data[org_data["total_counts"]<7500]

org_data.drop(["total_counts"],axis=1,inplace=True)
org_data.to_csv('complete_otus.csv',index=False)
# files.download('complete_otus.csv')

org_data


In [ ]:
le = preprocessing.LabelEncoder()

org_data_np=np.array(org_data)

labels=le.fit_transform(org_data_np[:,len(org_data_np[0])-1])

org_data_np[:,len(org_data_np[0])-1]=labels

stan_data=pd.DataFrame(org_data_np,columns=org_data.columns,index=org_data.index)
stan_data

In [ ]:
l_cols=taxonomy_arr
print(l_cols)
print(len(l_cols))
for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  print(temp)

**Data With columns as Phylum, Class, Order, Family, Genus**

**Phylum Wise Analysis**

In [ ]:
phyllum_labels=[]
count_no_phyllum=0

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)!=1):
    if (temp[1] not in phyllum_labels):
      phyllum_labels.append(temp[1])

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)==1):
    count_no_phyllum=count_no_phyllum+1

print("Number of Taxa without the phyllum labels:-",count_no_phyllum)

print("No. of distinct phyllum are:-",len(phyllum_labels))
print(phyllum_labels)

In [ ]:
numpy_data=np.array(stan_data)
print(l_cols)
stan_data

In [ ]:
temp_dict={}
for i in range(len(phyllum_labels)):
  temp_dict[phyllum_labels[i]]=0

temp_dict

In [ ]:
temps_arr=[]

for i in range(len(numpy_data)):
  temp_arr=numpy_data[i,0:len(numpy_data[0])-1].ravel()
  new_dict=copy.deepcopy(temp_dict)
  for j in range(len(temp_arr)):
    temp=l_cols[j]
    temp=temp.split(";")
    if (len(temp)>1):
      new_dict[temp[1]]=new_dict[temp[1]]+temp_arr[j]
  temps_arr.append(list(new_dict.values()))

temps_arr=np.array(temps_arr)

temp_columns=list(temp_dict.keys())

phylum_data=pd.DataFrame(temps_arr,columns=np.array(temp_columns),index=np.array(stan_data.index))
# phylum_data['treatment']=list(stan_data["treatment"])
print(temps_arr)

In [ ]:
phylum_data.to_csv("phylum_data.csv")
phylum_data

**Class Wise Analysis**

In [ ]:
class_labels=[]
count_no_class=0

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)>2):
    if (temp[2] not in class_labels):
      class_labels.append(temp[2])

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)==2):
    count_no_class=count_no_class+1

print("Number of Taxa without the class labels:-",count_no_class)

print("No. of distinct class are:-",len(class_labels))
print(class_labels)

In [ ]:
temp_dict={}
for i in range(len(class_labels)):
  temp_dict[class_labels[i]]=0

temp_dict

In [ ]:
numpy_data=np.array(stan_data)
print(l_cols)
stan_data

In [ ]:
temps_arr=[]

for i in range(len(numpy_data)):
  temp_arr=numpy_data[i,0:len(numpy_data[0])-1].ravel()
  new_dict=copy.deepcopy(temp_dict)
  for j in range(len(temp_arr)):
    temp=l_cols[j]
    temp=temp.split(";")
    if (len(temp)>2):
      new_dict[temp[2]]=new_dict[temp[2]]+temp_arr[j]
  temps_arr.append(list(new_dict.values()))

temps_arr=np.array(temps_arr)

temp_columns=list(temp_dict.keys())

class_data=pd.DataFrame(temps_arr,columns=np.array(temp_columns),index=np.array(stan_data.index))
# phylum_data['treatment']=list(stan_data["treatment"])
print(temps_arr)

In [ ]:
class_data.to_csv("class_data.csv")
class_data

**Order Wise Analysis**

In [ ]:
order_labels=[]
count_no_order=0

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)>3):
    if (temp[3] not in order_labels):
      order_labels.append(temp[3])

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)==3):
    count_no_order=count_no_order+1

print("Number of Taxa without the order labels:-",count_no_order)

print("No. of distinct order are:-",len(order_labels))
print(order_labels)

In [ ]:
temp_dict={}
for i in range(len(order_labels)):
  temp_dict[order_labels[i]]=0

temp_dict

In [ ]:
numpy_data=np.array(stan_data)
print(l_cols)
stan_data

In [ ]:
temps_arr=[]

for i in range(len(numpy_data)):
  temp_arr=numpy_data[i,0:len(numpy_data[0])-1].ravel()
  new_dict=copy.deepcopy(temp_dict)
  for j in range(len(temp_arr)):
    temp=l_cols[j]
    temp=temp.split(";")
    if (len(temp)>3):
      new_dict[temp[3]]=new_dict[temp[3]]+temp_arr[j]
  temps_arr.append(list(new_dict.values()))

temps_arr=np.array(temps_arr)

temp_columns=list(temp_dict.keys())

order_data=pd.DataFrame(temps_arr,columns=np.array(temp_columns),index=np.array(stan_data.index))
# phylum_data['treatment']=list(stan_data["treatment"])
print(temps_arr)

In [ ]:
order_data.to_csv("order_data.csv")
order_data

**Family Wise Analysis**

In [ ]:
family_labels=[]
count_no_family=0

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)>4):
    if (temp[4] not in family_labels):
      family_labels.append(temp[4])

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)==4):
    count_no_family=count_no_family+1

print("Number of Taxa without the order labels:-",count_no_family)

print("No. of distinct order are:-",len(family_labels))
print(family_labels)

In [ ]:
temp_dict={}
for i in range(len(family_labels)):
  temp_dict[family_labels[i]]=0

temp_dict

In [ ]:
numpy_data=np.array(stan_data)
print(l_cols)
stan_data

In [ ]:
temps_arr=[]

for i in range(len(numpy_data)):
  temp_arr=numpy_data[i,0:len(numpy_data[0])-1].ravel()
  new_dict=copy.deepcopy(temp_dict)
  for j in range(len(temp_arr)):
    temp=l_cols[j]
    temp=temp.split(";")
    if (len(temp)>4):
      new_dict[temp[4]]=new_dict[temp[4]]+temp_arr[j]
  temps_arr.append(list(new_dict.values()))

temps_arr=np.array(temps_arr)

temp_columns=list(temp_dict.keys())

family_data=pd.DataFrame(temps_arr,columns=np.array(temp_columns),index=np.array(stan_data.index))
# phylum_data['treatment']=list(stan_data["treatment"])
print(temps_arr)

In [ ]:
family_data.to_csv("family_data.csv")
family_data

**Genus Wise Analysis**

In [ ]:
genus_labels=[]
count_no_genus=0

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)>5):
    if (temp[5] not in genus_labels):
      genus_labels.append(temp[5])

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)==5):
    count_no_genus=count_no_genus+1

print("Number of Taxa without the order labels:-",count_no_genus)

print("No. of distinct order are:-",len(genus_labels))
print(genus_labels)

In [ ]:
temp_dict={}
for i in range(len(genus_labels)):
  temp_dict[genus_labels[i]]=0

temp_dict

In [ ]:
numpy_data=np.array(stan_data)
print(l_cols)
stan_data


In [ ]:
temps_arr=[]

for i in range(len(numpy_data)):
  temp_arr=numpy_data[i,0:len(numpy_data[0])-1].ravel()
  new_dict=copy.deepcopy(temp_dict)
  for j in range(len(temp_arr)):
    temp=l_cols[j]
    temp=temp.split(";")
    if (len(temp)>5):
      new_dict[temp[5]]=new_dict[temp[5]]+temp_arr[j]
  temps_arr.append(list(new_dict.values()))

temps_arr=np.array(temps_arr)

temp_columns=list(temp_dict.keys())

genus_data=pd.DataFrame(temps_arr,columns=np.array(temp_columns),index=np.array(stan_data.index))
# phylum_data['treatment']=list(stan_data["treatment"])
print(temps_arr)

In [ ]:
genus_data.to_csv("genus_data.csv")
genus_data

**Checking the Data**

In [ ]:
temp_all_data=[]
for i in range(len(numpy_data)):
  temp_all_data.append(sum(numpy_data[i,:]))

In [ ]:
genus_data_np=np.array(genus_data)

temp_genus_data=[]
for i in range(len(genus_data_np)):
  temp_genus_data.append(sum(genus_data_np[i,:]))


print(temp_all_data)
print(temp_genus_data)

for i in range(len(temp_all_data)):
  if (temp_all_data[i]<temp_genus_data[i]):
    print(True)

**Combining all the data**

In [ ]:
phylum_data_np=np.array(phylum_data)
class_data_np=np.array(class_data)
order_data_np=np.array(order_data)
family_data_np=np.array(family_data)
genus_data_np=np.array(genus_data)

In [ ]:
temp_data=np.c_[phylum_data_np,class_data_np,order_data_np,family_data_np,genus_data_np]

print(np.shape(temp_data))

comp_data=pd.DataFrame(temp_data,index=phylum_data.index,columns=list(phylum_data.columns)+list(class_data.columns)+list(order_data.columns)+list(family_data.columns)+list(genus_data.columns))
# comp_data["label"]=stan_data["label"]
comp_data.insert(loc=0, column='label', value=0.01)
comp_data.insert(loc=1, column='group', value=list(comp_data.index))
comp_data.insert(loc=2, column='count', value=359)

comp_data["treatment"]=stan_data['label']
comp_data.to_csv("comp_data.csv")
comp_data

**Lefse Plots**

In [ ]:
data_1=pd.read_csv("/content/Data_For_Biomarker_Analysis/Lefse_Analysis/ALL_Cats/output_ALL_Cats.csv")
data_1.dropna(inplace=True)
data_1.drop(["Unnamed: 0"],axis=1,inplace=True)
data_1

In [ ]:
temp_data=[]
data_np=np.array(data_1)
temp_labels=[]
for i in range(len(data_1)):
  if (data_np[i,1]<0):
    temp_labels.append("Adenocarcinoma")
  else:
    temp_labels.append("Squamous cell carcinoma")

data_1["Class"]=temp_labels

temp_data.append(data_1)
data_1

In [ ]:
fig = px.bar(data_1, x='scores', y='Names',orientation='h',color="Class",labels=dict(Names="", scores="LDA Score", Class="Groups"))
pio.write_image(fig, 'plot_all.svg',scale=7, width=800, height=600)
fig.show()

In [ ]:
Lda_threshold=0.5
# Lda_threshold_1=0.5

data_2=data_1[abs(data_1["scores"])>=Lda_threshold]

# data_1_temp_1 = data_1[data_1["Class"]=="Squamous"]
# data_1_temp_1 = data_1_temp_1[abs(data_1_temp_1["scores"])>=Lda_threshold_1]

# data_1_temp_2 = data_1[data_1["Class"]=="Adenocarcinoma"]
# data_1_temp_2 = data_1_temp_2[abs(data_1_temp_2["scores"])>=Lda_threshold]

# data_2 = pd.concat([data_1_temp_2,data_1_temp_1],axis=0)


In [ ]:
Class="Lung cancer subtypes"

In [ ]:
fig = px.bar(data_2, x='scores', y='Names',orientation='h',color="Class",
             hover_name="Class", color_discrete_sequence=["magenta", "orange"],
             labels=dict(Names="", scores="LDA Score"))

#pio.write_image(fig, 'plot_all_2.svg',scale=7, width=600, height=600)

fig.update_layout(
    autosize=False,
    width=700,
    height=800,
    font_color="black",
   )


#fig5.update_layout(legend=dict(
# orientation="h",entrywidth=85,font=dict(size=8)))
fig.show()


fig.write_image("lefse_otu.pdf")
fig.write_image("lefse_otu.svg")

fig.write_image("lefse_otu.png")


In [ ]:
fig = px.bar(data_2, x='scores', y='Names',orientation='h',color="Class",labels=dict(Names="", scores="LDA Score", Class="Groups"))
pio.write_image(fig, 'plot_all_2.svg',scale=7, width=600, height=600)
fig.show()

In [ ]:
temp_arr_names=[]

chosen_taxa=list(data_2["Names"])

print("Number of statistically significant taxa are:-",len(chosen_taxa))

for i in range(len(chosen_taxa)):
  if (str(chosen_taxa[i])=='g__0319.6G20'):
    temp_arr_names.append(" g__0319-6G20")
  elif (str(chosen_taxa[i])=='f__0319.6G20'):
    temp_arr_names.append(" f__0319-6G20")
  elif (str(chosen_taxa[i])=='o__0319.6G20'):
    temp_arr_names.append(" o__0319-6G20")
  else:
    temp_arr_names.append(" "+str(chosen_taxa[i]))

print(temp_arr_names)

In [ ]:
import copy
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
selected_feature_data=comp_data[temp_arr_names]
selected_feature_data_copy = copy.deepcopy(selected_feature_data)
selected_feature_data
font_size=5

fig=plt.figure(figsize=(8,8))


sns.heatmap(selected_feature_data.corr(),cmap="BrBG",annot=False)


#plt.savefig("lda_corr_plot.svg")
#plt.savefig("lda_corr_plot.png")
#plt.savefig("lda_corr_plot.pdf")

In [ ]:
selected_feature_data=comp_data[temp_arr_names]
selected_feature_data_copy = copy.deepcopy(selected_feature_data)
selected_feature_data

# Set the font size for the x and y labels
label_font_size = 6  # Change this to your desired font size

# Create the heatmap
fig = plt.figure(figsize=(7.5, 6.5))
ax = sns.heatmap(selected_feature_data.corr(), cmap="cividis", annot=False)

# Change the font size of x and y axis labels
ax.set_xticklabels(ax.get_xticklabels(), size=label_font_size)
ax.set_yticklabels(ax.get_yticklabels(), size=label_font_size)

# Show the plot
#plt.savefig("lda_corr.svg")
plt.show()

# Save the plot to an image file with a larger size
# Adjust the DPI and format as needed
ax.get_figure().savefig("heatmap.svg", dpi=600, bbox_inches='tight', format='svg')
ax.get_figure().savefig("heatmap.png", dpi=600, bbox_inches='tight', format='png')


In [ ]:
selected_feature_data.corr()


In [ ]:
import pandas as pd

# Assuming you have already defined your 'selected_feature_data' DataFrame and calculated the correlation matrix
correlation_matrix = selected_feature_data.corr()

# Save the correlation matrix to an Excel file (in this example, we're using XLSX format)
correlation_matrix.to_excel('correlation_matrix.xlsx', sheet_name='Correlation')


In [ ]:
# Create correlation matrix
corr_matrix = selected_feature_data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]

# Drop features
selected_feature_data.drop(to_drop, axis=1, inplace=True)

In [ ]:
# selected_feature_data.drop([" g__uncultured"],axis=1,inplace=True)
selected_feature_data[" g__Thermus"] = selected_feature_data_copy[" g__Thermus"]
selected_feature_data

In [ ]:
print("Old columns:-")
print(list(selected_feature_data_copy.columns))
print("New columns:-")
print(list(selected_feature_data.columns))

**Including Meta Data**

In [ ]:
meta_data=pd.read_csv("/content/Data_For_Biomarker_Analysis/Main_Data/Meta_Data.tsv",sep='\t')
meta_data.drop([0],axis=0,inplace=True)
meta_data

**Choosing the important columns**

In [ ]:
print(meta_data.columns)

**Choosing env_material, Host_Age, host_sex, smoker**

In [ ]:
# [" Sample-id","env_material","Host_Age","host_sex","smoker"]
meta_data=meta_data[[" Sample-id","env_material","Host_Age","host_sex","smoker"]]
meta_data

In [ ]:
meta_data_np=np.array(meta_data)
print(meta_data_np)

temp_sample_dict={}

for i in range(len(meta_data_np)):
  temp_sample_dict[meta_data_np[i,0]]=meta_data_np[i,1:]

In [ ]:
temp_sample_dict

In [ ]:
temp_sample_names=list(selected_feature_data.index)

chosen_meta_data_np=[]

for i in range(len(temp_sample_names)):
  chosen_meta_data_np.append(temp_sample_dict[temp_sample_names[i]])

chosen_meta_data_np=np.array(chosen_meta_data_np)
chosen_meta_data_np

chosen_meta_data=pd.DataFrame(chosen_meta_data_np,index=temp_sample_names,columns=list(meta_data.columns)[1:])
chosen_meta_data

**For the Data Labels**

In [ ]:
stan_data_np=np.array(stan_data)
list_sample_names=list(stan_data.index)

temp_sample_dict={}

for i in range(len(stan_data_np)):
  temp_sample_dict[list_sample_names[i]]=stan_data_np[i,len(stan_data_np[0])-1]



In [ ]:
temp_sample_dict

**Combining Meta data with taxa data**

In [ ]:
chosen_column1_np=np.array(selected_feature_data)
chosen_column2_np=np.array(chosen_meta_data)

data=np.array(stan_data)

x_data=np.c_[chosen_column1_np,chosen_column2_np]
print(np.shape(x_data))


result_data=pd.DataFrame(x_data,index=selected_feature_data.index,columns=list(selected_feature_data.columns)+list(chosen_meta_data.columns))

temp_na_data = copy.deepcopy(result_data)

#For seeing the Na elements
list_names_selected_feature_data=list(temp_na_data.index)
y_data_temp=[]
for i in range(len(list_names_selected_feature_data)):
  y_data_temp.append(temp_sample_dict[list_names_selected_feature_data[i]])

temp_na_data["label"]=y_data_temp


print(temp_na_data[temp_na_data.isnull().any(axis=1)])

#Removing The Na
result_data.dropna(inplace=True)

le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()

result_data["env_material"]=le1.fit_transform(result_data["env_material"])
result_data["host_sex"]=le2.fit_transform(result_data["host_sex"])
result_data["smoker"]=le3.fit_transform(result_data["smoker"])



result_data

In [ ]:
# temp_arr_taxa=[" p__Deinococcota"," c__Deinococci"," o__Thermales"," g__Thermus"]

# for i in range(len(temp_arr_taxa)):
#   temp=list(result_data[temp_arr_taxa[i]])
#   print(temp)

In [ ]:
list_names_selected_feature_data=list(result_data.index)
y_data=[]
for i in range(len(list_names_selected_feature_data)):
  y_data.append(temp_sample_dict[list_names_selected_feature_data[i]])

result_data["label"]=y_data

In [ ]:
print(le1.inverse_transform([0,1]))
print(le2.inverse_transform([0,1]))
print(le3.inverse_transform([0,1,2]))


In [ ]:
stan_data_np=np.array(stan_data).astype(int)
print("Number of features are:-",len(stan_data_np[0])-1)
print("Total Number of Instances are:-",len(+stan_data_np))

result_data_np=np.array(result_data).astype(float)
print("Number of features are:-",len(result_data_np[0])-1)
print("Total Number of Instances are:-",len(result_data_np))

In [ ]:
print("0-->",le.inverse_transform([0]),"1-->",le.inverse_transform([1]))

In [ ]:
stan_data

In [ ]:
result_data

**Exploratory Data Analysis**

In [ ]:
org_data['label'].value_counts().plot(kind='pie', autopct = "%1.0f%%")
plt.show()

**Analysing types**

In [ ]:
result_data=result_data.astype(float)
result_data_np.dtype

In [ ]:
stan_data_np.dtype

**Data Description**

In [ ]:
def my_df_describe(df):
    objects = []
    numerics = []
    for c in df:
        if (df[c].dtype == object):
            objects.append(c)
        else:
            numerics.append(c)

    return df[numerics].describe(), df[objects].describe()

**Correlation Matrix**

In [ ]:
temp_data=result_data.astype(int)
px.imshow(temp_data.corr())

**Outlier Analysis**

In [ ]:
# fig = px.box(temp_data,y=' f__Thermaceae',x="env_material",title=f"Distrubution of Age")
# fig.show()

In [ ]:
def cat_of_list(l):
  l1=[]
  for i in l:
    if i not in l1:
      l1.append(i)
  return l1

def categories_output(d):
  l_col=list(d.columns)
  l1=[]
  d1={}
  for i in l_col:
    l=list(d[i])
    if (type(l[0])==str):
      l1.append(i)
      d1[i]=cat_of_list(l)
  print(d1)
  return d1;

def visualisation_bar_hist(data):
  l_col=list(data.columns)
  temp_cat=categories_output(data)
  temp_keys=list(temp_cat.keys())
  temp_values=list(temp_cat.values())

  for i in range(len(l_col)-1):
    if l_col[i] in temp_keys:
      label=l_col[i]
      temp_val=temp_cat[label]
      temp_list=[]
      temp_columns=list(data[label])
      for j in range(len(temp_val)):
        temp_list.append(temp_columns.count(temp_val[j]))

      plt.bar(temp_val,temp_list,color='red')
      plt.title(l_col[i]+"-Bar Graph")
      plt.ylabel("Frequency")
      plt.show()
      print()
    else:
      temp_columns=list(data[l_col[i]])
      sns.histplot(temp_columns,color="red",kde=True)
      plt.title(l_col[i])
      plt.ylabel("Frequency")
      plt.show()
      print()

# visualisation_bar_hist(result_data)


In [ ]:
l_cols_result_data=list(result_data.columns)[0:len(result_data_np[0])-3]

result_data_no_out=copy.deepcopy(result_data)

# for i in range(len(l_cols_result_data)):
#   temp_=list(result_data_no_out[l_cols_result_data[i]])
#   # print(max(temp_))
#   result_data_no_out=result_data_no_out[result_data_no_out[l_cols_result_data[i]]<0.95*(max(temp_))]

In [ ]:
result_data_no_out

In [ ]:
#This function will basically give all the pairs of highly correlated features
def find_outliers(cov_data):
  l_cols=list(cov_data.columns)
  max_cov=-1
  in1=0
  in2=0
  for i in range(len(l_cols)):
    temp_l=list(cov_data[l_cols[i]])
    for j in range(len(l_cols)):
      if (max_cov<abs(temp_l[j]) and i!=j):
        in1=i
        in2=j

  return [l_cols[in1],l_cols[in2]]

#This is the x_data in pandas format
# x_data_pd=pd.DataFrame(x_data,columns=l_col_new[0:len(l_col_new)-1])
x_data_pd=result_data_no_out.drop(['label'],axis=1)

temp_outliers=find_outliers(x_data_pd.cov())

temp_data=x_data_pd.copy(deep=True)

for i in range(1):
  temp_outliers=find_outliers(temp_data.cov())
  plt.scatter(x_data_pd[temp_outliers[0]],x_data_pd[temp_outliers[1]],color="brown")
  plt.xlabel(temp_outliers[0])
  plt.ylabel(temp_outliers[1])
  temp_data=temp_data.drop([temp_outliers[0],temp_outliers[1]],axis=1)
  print("\n")
  plt.show()


**Data Preprocessing**

In [ ]:
x_data_pd=result_data_no_out.drop(['label'],axis=1)

#Normalisation
scaler = preprocessing.MinMaxScaler()

x_data_pd_stan=pd.DataFrame(scaler.fit_transform(x_data_pd),columns=x_data_pd.columns,index=x_data_pd.index)

x_data_pd_stan= x_data_pd

x_data_pd_stan["sample_id"] = result_data_no_out.index

x_data_pd['label']=result_data['label']

x_data_pd_stan['label']=result_data['label']


result_data=x_data_pd_stan


result_data_class0=result_data[result_data["label"]==0]
result_data_class1=result_data[result_data["label"]==1]

result_data_class_0_np=np.array(result_data_class0)
result_data_class_1_np=np.array(result_data_class1)

print(np.unique(result_data_class_0_np[:,len(result_data_class_0_np[0])-1]))
print(np.unique(result_data_class_1_np[:,len(result_data_class_1_np[0])-1]))

x_class_0=result_data_class_0_np[:,0:len(result_data_class_0_np[0])-1]
y_class_0=result_data_class_0_np[:,len(result_data_class_0_np[0])-1].astype(int)

x_class_1=result_data_class_1_np[:,0:len(result_data_class_1_np[0])-1]
y_class_1=result_data_class_1_np[:,len(result_data_class_1_np[0])-1].astype(int)

#CLASS 0
x_train_0, x_test_0, y_train_0, y_test_0 = train_test_split(x_class_0, y_class_0, test_size=0.3,shuffle=True, random_state=42)

#CLASS 1
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_class_1, y_class_1, test_size=0.3,shuffle=True, random_state=42)

#Train Data
x_train=np.r_[x_train_0,x_train_1]
y_train=np.r_[y_train_0,y_train_1]

sample_id_train = x_train[:,len(x_train[0])-1]

x_train = x_train[:,:len(x_train[0])-1].astype(float)



#Test Data
x_test=np.r_[x_test_0,x_test_1]
y_test=np.r_[y_test_0,y_test_1]

sample_id_test = x_test[:,len(x_test[0])-1]

x_test = x_test[:,:len(x_test[0])-1].astype(float)


print("Number of Instances in Training Data of class 0:-",len(x_train_0))
print("Number of Instances in Training Data of class 1:-",len(x_train_1))

print("Number of Instances in Testing Data of class 0:-",len(x_test_0))
print("Number of Instances in Testing Data of class 1:-",len(x_test_1))


**Applying LDA to the DATA**

In [ ]:
lda =LinearDiscriminantAnalysis().fit(x_train,y_train)

x_train = lda.transform(x_train)
x_test = lda.transform(x_test)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.scatter(x_train,y_train.reshape(-1,1),c=y_train.reshape(-1,1))
plt.savefig("lda_plot.pdf",dpi=1200)
plt.savefig("lda_plot.svg")
plt.show()

In [ ]:
plt.scatter(x_train[y_train==0],y_train.reshape(-1,1)[y_train==0],c='magenta',label="AC")
plt.scatter(x_train[y_train==1],y_train.reshape(-1,1)[y_train==1],c='orange',label="SCC")
plt.legend()
#plt.ylabel("Class Label")
plt.savefig("lda_before.pdf",dpi=1200)
plt.savefig("lda_before.svg")
plt.savefig("lda_before.png")
plt.show()

In [ ]:
plt.scatter(x_train,np.zeros((len(x_train),1)),c=y_train.reshape(-1,1),alpha=0.5)
plt.savefig("lda_after.svg")
plt.show()





In [ ]:
plt.scatter(x_train[y_train==0],np.zeros((len(x_train[y_train==0]),1)),c='magenta',label="AC")
plt.scatter(x_train[y_train==1],np.zeros((len(x_train[y_train==1]),1)),c='orange',label="SCC")
plt.legend()
plt.savefig("lda_after.pdf",dpi=1200)
plt.savefig("lda_after.svg")
plt.savefig("lda_after.png")
plt.show()

In [ ]:
class_0_train_samples = x_train[y_train==0]
class_1_train_samples = x_train[y_train==1]


sns.histplot({'AC': class_0_train_samples.ravel(), 'SCC': class_1_train_samples.ravel()}, bins=70, palette=['magenta', 'orange'], kde=True, stat='density')
plt.savefig("lda hist.png")
plt.show()



**Model Training**

**Logistic Regression**

In [ ]:
model_lr = LogisticRegression(random_state=0).fit(x_train,y_train)
y_pred_lr=model_lr.predict(x_test)

print("Training Accuracy is:-",model_lr.score(x_train,y_train))
print("Classification Accuracy is:-",model_lr.score(x_test,y_test))

In [ ]:
temp_arr=model_lr.coef_
print(temp_arr)
temp_arr.sort()
temp_arr

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_lr, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_lr,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_lr.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Gaussian Naive Bayes**

In [ ]:
model_nb = GaussianNB().fit(x_train,y_train)
y_pred_nb=model_nb.predict(x_test)

print("Training Accuracy is:-",model_nb.score(x_train,y_train))
print("Classification Accuracy is:-",model_nb.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_nb, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_nb,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_nb.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
skf = StratifiedKFold(n_splits=5)

**Decision Tree Classifier**

In [ ]:
def objective(trial):

  temp_acc = []

  max_depth = trial.suggest_int("max_depth", 1, 10)

  min_samples_split = trial.suggest_int("min_samples_split",2,50)

  min_samples_leaf = trial.suggest_int("min_samples_leaf",1,50)

  max_leaf_nodes = trial.suggest_int("max_leaf_nodes",5,100)

  #K-fold stratified Split
  for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
    y_train_temp = y_train[train_index]
    x_train_temp = x_train[train_index]

    y_val_temp = y_train[test_index]
    x_val_temp = x_train[test_index]

    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_leaf_nodes = max_leaf_nodes,random_state=42)

    #Training the Model
    model.fit(x_train_temp,y_train_temp.ravel())

    y_pred = model.predict(x_val_temp)

    v0 = metrics.accuracy_score(y_val_temp, y_pred)

    if (len(temp_data_np)==0):
      v0 = 0

    temp_acc.append(v0)

  return np.array(temp_acc).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print(study.best_params)


In [ ]:
temp_dict = study.best_params

In [ ]:
max_depth = temp_dict["max_depth"]

min_samples_split = temp_dict["min_samples_split"]

min_samples_leaf = temp_dict["min_samples_leaf"]

max_leaf_nodes = temp_dict["max_leaf_nodes"]

In [ ]:
model_dc = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_leaf_nodes = max_leaf_nodes,random_state=42).fit(x_train,y_train)
y_pred_dc=model_dc.predict(x_test)

print("Training Accuracy is:-",model_dc.score(x_train,y_train))
print("Classification Accuracy is:-",model_dc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_dc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_dc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_dc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Random Forest Classifier**

In [ ]:
def objective(trial):

  temp_acc = []

  max_depth = trial.suggest_int("max_depth", 1, 10)

  min_samples_split = trial.suggest_int("min_samples_split",2,50)

  min_samples_leaf = trial.suggest_int("min_samples_leaf",1,50)

  max_leaf_nodes = trial.suggest_int("max_leaf_nodes",5,100)

  n_estimators = trial.suggest_int("n_estimators",10,100)

  #K-fold stratified Split
  for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
    y_train_temp = y_train[train_index]
    x_train_temp = x_train[train_index]

    y_val_temp = y_train[test_index]
    x_val_temp = x_train[test_index]

    model = RandomForestClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_leaf_nodes = max_leaf_nodes,n_estimators=n_estimators,random_state=42)

    #Training the Model
    model.fit(x_train_temp,y_train_temp.ravel())

    y_pred = model.predict(x_val_temp)

    v0 = metrics.accuracy_score(y_val_temp, y_pred)

    temp_acc.append(v0)

  return np.array(temp_acc).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print(study.best_params)


In [ ]:
temp_dict = study.best_params

In [ ]:
max_depth = temp_dict["max_depth"]

min_samples_split = temp_dict["min_samples_split"]

min_samples_leaf = temp_dict["min_samples_leaf"]

max_leaf_nodes = temp_dict["max_leaf_nodes"]

n_estimators = temp_dict["n_estimators"]

In [ ]:
model_rfc = RandomForestClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, max_leaf_nodes = max_leaf_nodes,n_estimators=n_estimators,random_state=42).fit(x_train,y_train)
y_pred_rfc=model_rfc.predict(x_test)


print("Training Accuracy is:-",model_rfc.score(x_train,y_train))
print("Classification Accuracy is:-",model_rfc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_rfc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_rfc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_rfc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**XGBoost Classifier**

In [ ]:
def objective(trial):

  temp_acc = []

  max_depth = trial.suggest_int("max_depth", 1, 10)

  #K-fold stratified Split
  for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
    y_train_temp = y_train[train_index]
    x_train_temp = x_train[train_index]

    y_val_temp = y_train[test_index]
    x_val_temp = x_train[test_index]

    model = XGBClassifier(max_depth=max_depth,random_state=42)

    #Training the Model
    model.fit(x_train_temp,y_train_temp.ravel())

    y_pred = model.predict(x_val_temp)

    v0 = metrics.accuracy_score(y_val_temp, y_pred)
    temp_acc.append(v0)

  return np.array(temp_acc).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print(study.best_params)


In [ ]:
temp_dict = study.best_params

In [ ]:
max_depth = temp_dict["max_depth"]

In [ ]:
model_xgb = XGBClassifier(max_depth=max_depth,random_state=42).fit(x_train,y_train)
y_pred_xgb=model_xgb.predict(x_test)

print("Training Accuracy is:-",model_xgb.score(x_train,y_train))
print("Classification Accuracy is:-",model_xgb.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_xgb, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_xgb,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_xgb.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Gaussian Process Classifier**

In [ ]:
lbound = 1e-5
rbound = 1e5

n_restarts = 50

n_features = len(x_train[0])

# kernel=C(1.0, (lbound,rbound)) * RBF(n_features*[n_features], (lbound,rbound))  + WhiteKernel()

model_gpc = GaussianProcessClassifier().fit(x_train,y_train)
y_pred_gpc=model_gpc.predict(x_test)

print("Classification Accuracy is:-",model_gpc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_gpc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_gpc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_gpc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**KNN Classifier**

In [ ]:
def objective(trial):

  temp_acc = []

  n_neighbors = trial.suggest_int("n_neighbors", 1, 10)
  weights = trial.suggest_categorical("weights",["uniform", "distance"])
  p = trial.suggest_int("p", 1, 5)

  #K-fold stratified Split
  for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
    y_train_temp = y_train[train_index]
    x_train_temp = x_train[train_index]

    y_val_temp = y_train[test_index]
    x_val_temp = x_train[test_index]

    model = KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,p=p)

    #Training the Model
    model.fit(x_train_temp,y_train_temp.ravel())

    y_pred = model.predict(x_val_temp)

    v0 = metrics.accuracy_score(y_val_temp, y_pred)

    if (len(temp_data_np)==0):
      v0 = 0

    temp_acc.append(v0)

  return np.array(temp_acc).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print(study.best_params)


In [ ]:
temp_dict = study.best_params

In [ ]:
n_neighbors = temp_dict["n_neighbors"]
weights = temp_dict["weights"]
p = temp_dict["p"]

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=n_neighbors,weights=weights,p=p).fit(x_train,y_train)
y_pred_knn=model_knn.predict(x_test)

print("Classification Accuracy is:-",model_knn.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_knn, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_knn,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_knn.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**LDA Classifier**

In [ ]:
model_lda =LinearDiscriminantAnalysis().fit(x_train,y_train)
y_pred_lda=model_lda.predict(x_test)

print("Classification Accuracy is:-",model_lda.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_lda, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_lda,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_lda.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Support Vector Classifier**

In [ ]:
def objective(trial):

  temp_acc = []

  C = trial.suggest_float("C", 0, 1)

  kernel = trial.suggest_categorical("kernel",["linear", "poly", "rbf", "sigmoid"])

  #K-fold stratified Split
  for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
    y_train_temp = y_train[train_index]
    x_train_temp = x_train[train_index]

    y_val_temp = y_train[test_index]
    x_val_temp = x_train[test_index]

    model = SVC(probability=True, C=C, kernel=kernel,random_state = 42)

    #Training the Model
    model.fit(x_train_temp,y_train_temp.ravel())

    y_pred = model.predict(x_val_temp)

    v0 = metrics.accuracy_score(y_val_temp, y_pred)

    if (len(temp_data_np)==0):
      v0 = 0

    temp_acc.append(v0)

  return np.array(temp_acc).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print(study.best_params)


In [ ]:
temp_dict = study.best_params

In [ ]:
C = temp_dict["C"]

kernel = temp_dict["kernel"]

In [ ]:
model_svc = SVC(probability=True, C=C, kernel=kernel,random_state = 42).fit(x_train,y_train)
y_pred_svc=model_svc.predict(x_test)

print("Classification Accuracy is:-",model_svc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_svc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_svc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_svc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Sk-learn Neural Network Optimisation**

In [ ]:
def objective(trial):

  temp_acc = []

  n_layers = trial.suggest_int("n_layers", 1, 5)

  layers = []

  for i in range(n_layers):
    hidden_neuron = trial.suggest_int("Num Hidden_"+str(i), 1, 500)
    layers.append(hidden_neuron)

  #L2 Regularisation term
  alpha = trial.suggest_float("alpha",1e-5,1e-2)

  learning_rate = trial.suggest_categorical("learning_rate",["constant", "invscaling", "adaptive"])

  learning_rate_init = trial.suggest_float("learning_rate_init",1e-5,1e-1)

  max_iter = trial.suggest_int("max_iter", 50, 2000)

  tol = trial.suggest_float("tol",1e-5,1e-3)

  #K-fold stratified Split
  for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
    y_train_temp = y_train[train_index]
    x_train_temp = x_train[train_index]

    #Data for C-means
    x_train_data_temp = x_train_temp[:,0:len(x_train_temp[0])-4]
    x_train_meta_temp = x_train_temp[:,len(x_train_temp[0])-4:]

    y_val_temp = y_train[test_index]
    x_val_temp = x_train[test_index]


    model = MLPClassifier(hidden_layer_sizes=layers , alpha = alpha, learning_rate= learning_rate, learning_rate_init= learning_rate_init, max_iter= max_iter,tol=tol,early_stopping=True,validation_fraction=0.3 ,random_state=42)

    #Training the Model

    model.fit(x_train_temp,y_train_temp.ravel())

    y_pred = model.predict(x_val_temp)

    v0 = metrics.accuracy_score(y_val_temp, y_pred)

    if (len(temp_data_np)==0):
      v0 = 0

    temp_acc.append(v0)

  return np.array(temp_acc).mean()




study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

print(study.best_params)


In [ ]:
temp_dict = study.best_params

In [ ]:
n_layers = temp_dict["n_layers"]

layers = []

for i in range(n_layers):
  hidden_neuron = temp_dict["Num Hidden_"+str(i)]
  layers.append(hidden_neuron)


#L2 Regularisation term
alpha = temp_dict["alpha"]

learning_rate = temp_dict["learning_rate"]

learning_rate_init = temp_dict["learning_rate_init"]

max_iter = temp_dict["max_iter"]

tol = temp_dict["tol"]

model_nn = MLPClassifier(hidden_layer_sizes=layers , alpha = alpha, learning_rate= learning_rate, learning_rate_init= learning_rate_init, max_iter= max_iter,tol=tol,early_stopping=True,validation_fraction=0.3 ,random_state=42)



#Training the Model
model_nn.fit(x_train,y_train.ravel())


In [ ]:
y_pred_nn=model_nn.predict(x_test)

print("Accuracy on the training data is:-",model_nn.score(x_train,y_train))
print("Classification Accuracy is:-",model_nn.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_nn, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_nn,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
for i in range(len(y_pred_nn)):
  if (y_pred_nn[i]==0 and y_pred_nn[i]!=y_test[i]):
    print(sample_id_test[i])

In [ ]:
for i in range(len(y_pred_nn)):
  if (y_pred_nn[i]==1 and y_pred_nn[i]!=y_test[i]):
    print(sample_id_test[i])

In [ ]:
probs = model_nn.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()